In [1]:
import sys
import heapq
import itertools

# DIRECTION_MAP dict of possible directions based on ingress direction
DIRECTION_MAP = {
    'N': {'L': 'W', 'R': 'E', 'F': 'N'},
    'S': {'L': 'E', 'R': 'W', 'F': 'S'},
    'E': {'L': 'N', 'R': 'S', 'F': 'E'},
    'W': {'L': 'S', 'R': 'N', 'F': 'W'}
}


In [3]:
def position_iterator():
    return itertools.product(range(len(CITY_MAP)), range(len(CITY_MAP[0])))

In [5]:
def next_position(my_map, current_position, direction):
    next_pos = None
    # up
    if direction == 'N':
        if current_position[0] != 0:
            next_pos = (current_position[0] - 1, current_position[1])
    # down
    elif direction == 'S':
        if current_position[0] < len(my_map) - 1:
            next_pos = (current_position[0] + 1, current_position[1])
    # left
    elif direction == 'W':
        if current_position[1] != 0:
            next_pos = (current_position[0], current_position[1] - 1)
    # right
    elif direction == 'E':
        if current_position[1] < len(my_map[0]) - 1:
            next_pos = (current_position[0], current_position[1] + 1)
    return next_pos

In [7]:
def reconstruct_path(previous_nodes, min_heat_loss_key):
    #print(f"reconstruct_path(previous_nodes, {min_heat_loss_key})")
    #print(f"previous_nodes = {previous_nodes}")
    #print(f"min_heat_loss_key ={min_heat_loss_key}")
    #print(f"previous_nodes[min_heat_loss_key] = {previous_nodes.get(min_heat_loss_key)}")

    path = []
    current_node = min_heat_loss_key
    while current_node is not None:
        path.append(current_node)
        current_node = previous_nodes.get(current_node)
        #print(f"current_node: {current_node}")

    path.reverse()  # Reverse the path to get it from start to end
    return path
    

In [101]:
def dijkstra(heat_map, start, min_forward, max_forward):
    #print(f"dijkstra(heat_map, {start}, {min_forward}, {max_forward})")
    pq = [
        (0, start, 0, 'S'),
        (0, start, 0, 'E')
    ]  # (cost, position, forward movements, ingress direction)

    previous_blocks = {}
    heat_loss = {}
    heat_loss[(start, 0, 'S')] = 0
    heat_loss[(start, 0, 'E')] = 0

    while pq:
        current_heat_loss, current_position, forward_movements, direction = heapq.heappop(pq)
        current_position_key = (current_position, forward_movements, direction)
        
        if current_position_key not in previous_blocks:
            previous_blocks[current_position_key] = None

        # Early termination if we've reached the bottom-right corner
        if current_position == (len(heat_map) - 1, len(heat_map[0]) - 1):
            break

        for movement in 'LRF':
            new_direction = DIRECTION_MAP[direction][movement]
            new_position = next_position(heat_map, current_position, new_direction)
            new_forward_movements = forward_movements + 1 if movement == 'F' else 1

            if new_position:
                new_heat_loss = current_heat_loss + heat_map[new_position[0]][new_position[1]]
                
                # Check if this movement results in a lower heat loss
                if new_heat_loss < heat_loss.get((new_position, new_forward_movements, new_direction), float('infinity')):
                    # Skip forward movement if it exceeds max_forward
                    if movement == 'F' and new_forward_movements > max_forward:
                        continue
                    # Skip non-forward movement if not enough forward moves
                    if movement != 'F' and forward_movements < min_forward:
                        continue
                    
                    heat_loss[(new_position, new_forward_movements, new_direction)] = new_heat_loss
                    new_position_key = (new_position, new_forward_movements, new_direction)
                    previous_blocks[new_position_key] = current_position_key
                    #print(f"heapq.heappush(pq, ({new_heat_loss}, {new_position}, {new_forward_movements}, {new_direction}))")
                    heapq.heappush(pq, (new_heat_loss, new_position, new_forward_movements, new_direction))

    return heat_loss, previous_blocks

In [9]:
def build_map_graph(city_map):
    graph = {}
    return graph

In [11]:
def print_map(grid,path):
    rows = len(grid)
    cols = len(grid[0])
    grid_with_path = [[' ' for _ in range(cols)] for _ in range(rows)]
    symbol = {
        'N': '^',
        'S': 'v',
        'E': '>',
        'W': '<'

    }
    for r in range(rows):
        for c in range(cols):
            grid_with_path[r][c] = str(grid[r][c])
    
    for (r, c), moves, direction in path:
        grid_with_path[r][c] = symbol[direction]
    
    for row in grid_with_path:
        print(''.join(row)) 
    

In [111]:
# Load map as list of lists
with open("/Users/corey/dev/dijkstra/input.txt", 'r') as file:
    CITY_MAP = [[int(char) for char in line] for line in file.read().splitlines()]
# Start at top left
start = (0, 0)
# End at bottom right
end = (len(CITY_MAP) - 1, len(CITY_MAP[0]) - 1)


In [119]:
parts = {
    'part1': (0,3),
    'part2': (4,10)

}

In [123]:
for part, (min_moves, max_moves) in parts.items():
    print(f"{part}: {min_moves}, {max_moves}")
    heat_loss, previous_blocks = dijkstra(CITY_MAP, start, min_moves, max_moves)
    min_heat_loss_key = None
    for key, path_heat_loss in heat_loss.items():
        #print(key,path_heat_loss)
        (position, forward_movements, direction) = key    
        if position == end:
            if not min_heat_loss_key:
                min_heat_loss_key = key
            elif heat_loss[key] < heat_loss[min_heat_loss_key]:
                min_heat_loss_key = key
            #print(min_heat_loss_key, heat_loss[key])
            
    if min_heat_loss_key:
        print(f"Heat loss to {end}: {heat_loss[min_heat_loss_key]}")
    else:
        print(f"No path found to {end}")
    #print(f"previous_blocks = {previous_blocks}")
    #my_path = reconstruct_path(previous_blocks, min_heat_loss_key)
    #print(f"Path: {my_path}")
    #print_map(CITY_MAP,my_path)

part1: 0, 3
Heat loss to (140, 140): 870
part2: 4, 10
Heat loss to (140, 140): 1063
